# Transfer Learning: BERT

In [1]:
import torch
print(torch.__version__)


2.3.1+cpu


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import pipeline


c:\Users\drake\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# Load the cleaned and processed dataset
file_path = 'cleaned_movie_data.csv'
cleaned_data = pd.read_csv(file_path)

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(cleaned_data['processed_text'], cleaned_data['label'], test_size=0.2, random_state=42)

In [5]:
# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Tokenize the text data
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=128)

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, y_train.tolist())
test_dataset = SentimentDataset(test_encodings, y_test.tolist())

In [7]:
import transformers
import accelerate

print(transformers.__version__)
print(accelerate.__version__)


4.42.4
0.32.1


In [8]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=100,
    per_device_eval_batch_size=640,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [9]:
# Train the model
trainer.train()

  3%|▎         | 10/320 [07:23<3:49:24, 44.40s/it]

{'loss': 0.7956, 'grad_norm': 6.093982219696045, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.03}


  6%|▋         | 20/320 [14:36<3:33:35, 42.72s/it]

{'loss': 0.7336, 'grad_norm': 2.5426721572875977, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.06}


  9%|▉         | 30/320 [23:50<4:41:50, 58.31s/it]

...

100%|██████████| 320/320 [4:00:47<00:00, 43.42s/it]

{'loss': 0.4128, 'grad_norm': 8.834904670715332, 'learning_rate': 3.2000000000000005e-05, 'epoch': 1.0}


100%|██████████| 320/320 [4:00:51<00:00, 45.16s/it]


{'train_runtime': 14450.576, 'train_samples_per_second': 2.214, 'train_steps_per_second': 0.022, 'train_loss': 0.5008510336279869, 'epoch': 1.0}


TrainOutput(global_step=320, training_loss=0.5008510336279869, metrics={'train_runtime': 14450.576, 'train_samples_per_second': 2.214, 'train_steps_per_second': 0.022, 'total_flos': 2104888442880000.0, 'train_loss': 0.5008510336279869, 'epoch': 1.0})

In [10]:
# Evaluate the model
trainer.evaluate()

100%|██████████| 13/13 [15:06<00:00, 69.72s/it]


{'eval_loss': 0.37715137004852295,
 'eval_runtime': 984.2814,
 'eval_samples_per_second': 8.128,
 'eval_steps_per_second': 0.013,
 'epoch': 1.0}

In [14]:
max_length = 512

In [19]:
import pickle
# Save the model using pickle
with open('bert_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# Make predictions and evaluate the model
bert_pipeline = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
X_test_str = X_test.tolist()
predictions = bert_pipeline(X_test_str, truncation=True, padding=True, max_length=max_length)

# Convert predictions to labels
y_pred = [1 if pred['label'] == 'LABEL_1' else 0 for pred in predictions]

# Print the accuracy and classification report
print("BERT Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))